In [13]:
from enum import Enum
from dataclasses import dataclass, asdict

In [12]:
class DatasetTypes(str, Enum):
    simple = "simple"
    split = "split"
    diagonal = "diagonal"
    xor = "xor"

In [7]:
[name for (name, value) in DatasetTypes.__members__.items()]

['simple', 'split', 'diagonal', 'xor']

In [9]:
@dataclass
class Dataset:
    simple: int
    split: int

In [14]:
datasets = Dataset(5, 10)
asdict(datasets)[DatasetTypes.simple]

5